In [2]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D

Using TensorFlow backend.


In [3]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/CSE 240/Project/Data


In [4]:
%cd spectrogram_images/

/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_images


In [0]:
img = load_img('concentration/S001E02_0/S001E02_0_AF3.png', target_size=(29,43))

In [6]:
print(type(img))
print(img.format)
print(img.mode)
print(img.size)
print(img)

<class 'PIL.Image.Image'>
None
RGB
(43, 29)
<PIL.Image.Image image mode=RGB size=43x29 at 0x7F4351BAF978>


In [0]:
print(type(img))
# convert to numpy array
img_array = img_to_array(img)
print(img_array.dtype)
print(img_array.shape)
print(img_array[0][0])
# convert back to image
img_pil = array_to_img(img_array)
print(type(img))
#for i in range(288):
  #for j in range(432):
    #print(img_array[i][j])

<class 'PIL.Image.Image'>
float32
(288, 432, 3)
[255. 255. 255.]
<class 'PIL.Image.Image'>


In [0]:
import os
import numpy as np
import pandas as pd

In [0]:
base_path = "concentration"
concentration_data_array = []
for foldername in os.listdir(base_path):
  print(foldername)
  array = np.zeros((144,216,1))
  for filename in os.listdir(os.path.join(base_path, foldername)):
    example = foldername
    channel = filename.split('.')[0].split("_")[1]
    filepath = os.path.join(os.path.join(base_path, foldername), filename)
    #print(filename,filepath)
    img = load_img(filepath, target_size=(144,216))
    img_array = img_to_array(img)
    #print(img_array.shape, array.shape)
    array = np.concatenate((array, img_array), axis=2)
    #print(img_array.shape, array.shape)
  concentration_data_array.append(np.delete(array,0,2))
#print(data_array['S025E02'].shape)
concentration_data_array = np.array(concentration_data_array)
print(concentration_data_array.shape)
np.save("concentration_array_50.npy", concentration_data_array)

S015E02_8
S015E02_9
S015E02_10
S015E02_11
S015E02_12
S015E02_13
S015E02_14
S015E02_15
S015E02_16
S015E02_17
S015E02_18
S015E02_19
S015E02_20
S015E02_21
S015E02_22
S015E02_23
S015E02_24
S015E02_25
S015E02_26
S015E02_27
S015E02_28
S015E02_29
S015E02_30
S015E02_31
S015E02_32
S015E02_33
S015E02_34
S015E02_35
S015E04_0
S015E04_1
S015E04_2
S015E04_3
S015E04_4
S015E04_5
S015E04_6
S015E04_7
S015E04_8
S015E04_9
S015E04_10
S015E04_11
S015E04_12
S015E04_13
S015E04_14
S015E04_15
S015E04_16
S015E04_17
S015E04_18
S015E04_19
S015E04_20
S015E04_21
S015E04_22
S015E04_23
S015E04_24
S015E04_25
S015E04_26
S015E04_27
S015E04_28
S015E04_29
S015E04_30
S015E04_31
S015E04_32
S015E04_33
S015E04_34
S015E04_35
S017E02_0
S017E02_1
S017E02_2
S017E02_3
S017E02_4
S017E02_5
S017E02_6
S017E02_7
S017E02_8
S017E02_9
S017E02_10
S017E02_11
S017E02_12
S017E02_13
S017E02_14


Save to file


In [0]:
np.save("concentration_array.npy", concentration_data_array)

In [0]:
concentration_data_array = None
loaded_array = np.load("concentration_array.npy")
print(loaded_array.shape)

(2088, 29, 43, 42)


AttributeError: ignored

In [0]:
base_path = "relaxed"
relaxed_data_array = []
for foldername in os.listdir(base_path):
  print(foldername)
  array = np.zeros((29,43,1))
  for filename in os.listdir(os.path.join(base_path, foldername)):
    example = foldername
    channel = filename.split('.')[0].split("_")[1]
    filepath = os.path.join(os.path.join(base_path, foldername), filename)
    #print(filename,filepath)
    img = load_img(filepath, target_size=(29,43))
    img_array = img_to_array(img)
    #print(img_array.shape, array.shape)
    array = np.concatenate((array, img_array), axis=2)
    #print(img_array.shape, array.shape)
  relaxed_data_array.append(np.delete(array,0,2))
#print(data_array['S025E02'].shape)
relaxed_data_array = np.array(relaxed_data_array)
print(relaxed_data_array.shape)
np.save("relaxed_array.npy", relaxed_data_array)

S015E01_8
S015E01_9
S015E01_10
S015E01_11
S015E01_12
S015E01_13
S015E01_14
S015E01_15
S015E01_16
S015E01_17
S015E01_18
S015E01_19
S015E01_20
S015E01_21
S015E01_22
S015E01_23
S015E01_24
S015E01_25
S015E01_26
S015E01_27
S015E01_28
S015E01_29
S015E01_30
S015E01_31
S015E01_32
S015E01_33
S015E01_34
S015E01_35
S015E03_0
S015E03_1
S015E03_2
S015E03_3
S015E03_4
S015E03_5
S015E03_6
S015E03_7
S015E03_8
S015E03_9
S015E03_10
S015E03_11
S015E03_12
S015E03_13
S015E03_14
S015E03_15
S015E03_16
S015E03_17
S015E03_18
S015E03_19
S015E03_20
S015E03_21
S015E03_22
S015E03_23
S015E03_24
S015E03_25
S015E03_26
S015E03_27
S015E03_28
S015E03_29
S015E03_30
S015E03_31
S015E03_32
S015E03_33
S015E03_34
S015E03_35
S017E01_0
S017E01_1
S017E01_2
S017E01_3
S017E01_4
S017E01_5
S017E01_6
S017E01_7
S017E01_8
S017E01_9
S017E01_10
S017E01_11
S017E01_12
S017E01_13
S017E01_14
S017E01_15
S017E01_16
S017E01_17
S017E01_18
S017E01_19
S017E01_20
S017E01_21
S017E01_22
S017E01_23
S017E01_24
S017E01_25
S017E01_26
S017E01_27
S017E01_28

In [0]:
concentration_data_array = np.load("concentration_array.npy")
relaxed_data_array = np.load("relaxed_array.npy")

In [0]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
relaxed_data_array, concentration_data_array = None, None

In [0]:
X.shape

(4140, 29, 43, 42)

In [0]:

Y = np.concatenate((np.zeros((2052,1)),np.ones((2088,1))))
print(X.shape, Y.shape)

(4140, 29, 43, 42) (4140, 1)


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X, Y = None, None

(3519, 29, 43, 42) (621, 29, 43, 42) (3519, 1) (621, 1)


In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)

(2991, 29, 43, 42) (621, 29, 43, 42) (528, 29, 43, 42) (2991, 1) (621, 1) (528, 1)


In [0]:
print(X_train[4][200][0])

[255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.
 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255. 255.]


In [0]:
datagen = ImageDataGenerator()
train_base_path = "Spectrograms/train"
test_base_path = "Spectrograms/test"
validation_base_path = "Spectrograms/validation"
train_generator = datagen.flow_from_directory(
        train_base_path,
        target_size=(288, 432),
        class_mode='binary')
test_generator = datagen.flow_from_directory(
        test_base_path,
        target_size=(288, 432),
        class_mode='binary')
validation_generator = datagen.flow_from_directory(
        validation_base_path,
        target_size=(288, 432),
        class_mode='binary')

Found 1344 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Found 168 images belonging to 2 classes.


In [0]:
# confirm the iterator works
batchX, batchy = train_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))
batchX, batchy = validation_generator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batch shape=(32, 288, 432, 3), min=0.000, max=255.000
Batch shape=(32, 288, 432, 3), min=0.000, max=255.000


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D

#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(288, 432, 42)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [0]:
adam = optimizers.Adam(lr=0.00000001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=42, validation_data=validation_generator, validation_steps=6)

Epoch 1/10
42/42 [==============================] - 195s 5s/step - loss: 7.8170 - acc: 0.5097 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/10
42/42 [==============================] - 250s 6s/step - loss: 8.2796 - acc: 0.4807 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/10
42/42 [==============================] - 206s 5s/step - loss: 7.8051 - acc: 0.5104 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/10
42/42 [==============================] - 203s 5s/step - loss: 7.8763 - acc: 0.5060 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/10
42/42 [==============================] - 196s 5s/step - loss: 8.1610 - acc: 0.4881 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 6/10
42/42 [==============================] - 217s 5s/step - loss: 7.7814 - acc: 0.5119 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 7/10
42/42 [==============================] - 236s 6s/step - loss: 8.1373 - acc: 0.4896 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 8/10
42/42 [==============================] - 203s 5s/step - loss: 7.8170 - a

In [0]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 2/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 3/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 4/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 5/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 6/50
86/86 [==============================] - 3s 30ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 7/50
86/86 [==============================] - 3s 31ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 5.9784 - val_acc: 0.6250
Epoch 8/50
86/86 [=====================

In [0]:
# evaluate model
loss = model.evaluate_generator(test_generator, steps=6)

In [0]:
from keras import optimizers
#create model
model2 = Sequential()
#add model layers
model2.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(29, 43, 42)))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(32, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model2.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)




Train on 2991 samples, validate on 528 samples
Epoch 1/50





2991/2991 [==============================] - 19s 6ms/step - loss: 6.4773 - acc: 0.4754 - val_loss: 5.8327 - val_acc: 0.4811
Epoch 2/50
2991/2991 [==============================] - 5s 2ms/step - loss: 5.2788 - acc: 0.4804 - val_loss: 5.2367 - val_acc: 0.4678
Epoch 3/50
2991/2991 [==============================] - 5s 2ms/step - loss: 4.8965 - acc: 0.4865 - val_loss: 4.9559 - val_acc: 0.4716
Epoch 4/50
2991/2991 [==============================] - 5s 2ms/step - loss: 4.6621 - acc: 0.4928 - val_loss: 4.8091 - val_acc: 0.4773
Epoch 5/50
2991/2991 [==============================] - 5s 2ms/step - loss: 4.5020 - acc: 0.4985 - val_loss: 4.7007 - val_acc: 0.4792
Epoch 6/50
2991/2991 [==============================] - 5s 2ms/step - loss: 4.4084 - acc: 0.5022 - val_loss: 4.6369 - val_acc: 0.4792
Epoch 7/50
2991/2991 [==============================] - 5s 2ms/step - loss: 4.3325 - acc: 0.5048 - val_loss: 4.5804 - val_acc: 0.4905
Epoch 

In [0]:

model3 = Sequential()
#add model layers
model3.add(SeparableConv2D(512, kernel_size=5, activation='relu', input_shape=(29, 43, 42)))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Conv2D(64, kernel_size=3, activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Flatten())
model3.add(Dense(50, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model3.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 2991 samples, validate on 528 samples
Epoch 1/50
2991/2991 [==============================] - 10s 3ms/step - loss: 6.2329 - acc: 0.4888 - val_loss: 5.1691 - val_acc: 0.5114
Epoch 2/50
2991/2991 [==============================] - 8s 3ms/step - loss: 4.6302 - acc: 0.4888 - val_loss: 3.6184 - val_acc: 0.5114
Epoch 3/50
2991/2991 [==============================] - 8s 3ms/step - loss: 3.0519 - acc: 0.4881 - val_loss: 2.2074 - val_acc: 0.5114
Epoch 4/50
2991/2991 [==============================] - 8s 3ms/step - loss: 1.7548 - acc: 0.4838 - val_loss: 1.2699 - val_acc: 0.4716
Epoch 5/50
2991/2991 [==============================] - 8s 3ms/step - loss: 1.0833 - acc: 0.4691 - val_loss: 0.9718 - val_acc: 0.4659
Epoch 6/50
2991/2991 [==============================] - 8s 3ms/step - loss: 0.9074 - acc: 0.4637 - val_loss: 0.9239 - val_acc: 0.4621
Epoch 7/50
2991/2991 [==============================] - 9s 3ms/step - loss: 0.8718 - acc: 0.4748 - val_loss: 0.9132 - val_acc: 0.4697
Epoch 8/50
299

In [0]:
model4 = Sequential()
#add model layers
model4.add(SeparableConv2D(256, kernel_size=5, activation='relu', input_shape=(288, 432, 42)))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(SeparableConv2D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Flatten())
model4.add(Dense(50, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model4.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model4.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/50
86/86 [==============================] - 6s 75ms/step - loss: 1.1216 - acc: 0.5000 - val_loss: 1.0257 - val_acc: 0.6250
Epoch 2/50
86/86 [==============================] - 5s 60ms/step - loss: 1.0565 - acc: 0.5000 - val_loss: 0.9893 - val_acc: 0.6250
Epoch 3/50
86/86 [==============================] - 5s 60ms/step - loss: 0.9977 - acc: 0.5000 - val_loss: 0.9556 - val_acc: 0.6250
Epoch 4/50
86/86 [==============================] - 5s 60ms/step - loss: 0.9466 - acc: 0.5000 - val_loss: 0.9273 - val_acc: 0.6250
Epoch 5/50
86/86 [==============================] - 5s 60ms/step - loss: 0.9026 - acc: 0.5000 - val_loss: 0.9054 - val_acc: 0.6250
Epoch 6/50
86/86 [==============================] - 5s 60ms/step - loss: 0.8642 - acc: 0.5000 - val_loss: 0.8890 - val_acc: 0.6250
Epoch 7/50
86/86 [==============================] - 5s 60ms/step - loss: 0.8323 - acc: 0.4884 - val_loss: 0.8760 - val_acc: 0.6250
Epoch 8/50
86/86 [=====================

In [0]:
model5 = Sequential()
#add model layers
model5.add(Conv3D(256, kernel_size=3, activation='relu', input_shape=(288, 432, 42)))
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(Conv2D(64, kernel_size=3, activation='relu'))
model5.add(MaxPooling2D(pool_size=(2, 2)))
model5.add(Flatten())
model5.add(Dense(50, activation='relu'))
model5.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model5.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model5.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

ValueError: ignored

In [0]:
from sklearn import svm
model5 = svm.SVC()
model5.fit(X_train, y_train)

ValueError: ignored

In [0]:
model6 = Sequential()
#add model layers
model6.add(DepthwiseConv2D(kernel_size=(3,3), activation='relu', input_shape=(288, 432, 42)))
model6.add(MaxPooling2D(pool_size=(2, 2)))
model6.add(DepthwiseConv2D(kernel_size=(3,3), activation='relu'))
model6.add(MaxPooling2D(pool_size=(2, 2)))
model6.add(Flatten())
model6.add(Dense(50, activation='relu'))
model6.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.00000001, momentum=0.5)
model6.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model6.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=150, batch_size=8, shuffle=True)

Train on 86 samples, validate on 16 samples
Epoch 1/150
86/86 [==============================] - 5s 62ms/step - loss: 1.7861 - acc: 0.5000 - val_loss: 2.4542 - val_acc: 0.3750
Epoch 2/150
86/86 [==============================] - 4s 49ms/step - loss: 1.7211 - acc: 0.5000 - val_loss: 2.3723 - val_acc: 0.3750
Epoch 3/150
86/86 [==============================] - 4s 49ms/step - loss: 1.6565 - acc: 0.5000 - val_loss: 2.2836 - val_acc: 0.3750
Epoch 4/150
86/86 [==============================] - 4s 49ms/step - loss: 1.5939 - acc: 0.5000 - val_loss: 2.2029 - val_acc: 0.3750
Epoch 5/150
86/86 [==============================] - 4s 49ms/step - loss: 1.5336 - acc: 0.5000 - val_loss: 2.1223 - val_acc: 0.3750
Epoch 6/150
86/86 [==============================] - 4s 49ms/step - loss: 1.4753 - acc: 0.5000 - val_loss: 2.0438 - val_acc: 0.3750
Epoch 7/150
86/86 [==============================] - 4s 49ms/step - loss: 1.4188 - acc: 0.5000 - val_loss: 1.9698 - val_acc: 0.3750
Epoch 8/150
86/86 [=============

In [0]:
metrics = model2.evaluate(X_test,y_test)
print(metrics)

621/621 [==============================] - 0s 795us/step
[3.535203630029675, 0.5378421904000299]


In [0]:
metrics = model.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 0s 22ms/step
[9.742568969726562, 0.3888888955116272]


In [0]:
metrics = model3.evaluate(X_test,y_test)
print(metrics)

621/621 [==============================] - 1s 1ms/step
[0.7628745698698477, 0.5346215784837658]


In [0]:
metrics = model4.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 1s 31ms/step
[0.6619123220443726, 0.6111111044883728]


In [0]:
metrics = model6.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 0s 21ms/step
[0.8770436644554138, 0.3888888955116272]


In [0]:
# higher epochs version
metrics = model6.evaluate(X_test,y_test)
print(metrics)

18/18 [==============================] - 0s 22ms/step
[0.8298603892326355, 0.3888888955116272]
